In [2]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
import pandas as pd
from gensim.models import Phrases
from gensim import corpora, models
from nltk.tokenize import word_tokenize 
df=pd.read_csv("/content/sample_data/Final.csv")
tokens=[]
bigram = Phrases()
for a in df["cleaned_tweet"]:
  tokens.append(word_tokenize(a))
bigram = models.Phrases(tokens, min_count=30)
bigram_mod = models.phrases.Phraser(bigram)
# trigram =models.Phrases(bigram[tokens])  
# trigram_mod = models.phrases.Phraser(trigram)

In [0]:
#Top n-grams
import operator
dic={}
count=0
bigrams=[]
for id in range(len(tokens)):
    for token in bigram_mod[tokens[id]]:
        if '_' in token:
          bigrams.append(token)
          tokens[id].append(token)
for co in bigrams:
  if co in dic:
    dic[co]+=1
  else:
    dic[co]=1
bi= sorted(dic.items(),key=operator.itemgetter(1),reverse=True)


In [6]:
bi[:10]

[('test_posit', 16045),
 ('dure_pandem', 14673),
 ('stay_home', 10218),
 ('via_youtub', 9095),
 ('social_distanc', 8541),
 ('confirm_case', 7021),
 ('dure_crisi', 6959),
 ('public_health', 6193),
 ('new_york', 5972),
 ('stay_safe', 5353)]

In [0]:
# Remove rare and common tokens.
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(tokens)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=30, no_above=0.5)

In [0]:
#Dictionary Creation
# id2word = corpora.Dictionary(bigram_mod[tokens])
# Creating Bag-of-words
# corpus = [id2word.doc2bow(text) for text in bigram_mod[tokens]]
# print(corpus[:4])

# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in tokens]

In [9]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 22212
Number of documents: 997924


In [10]:
# Train LDA mode
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token
lda_model = models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           chunksize=2000,
                                           alpha='auto',
                                           eta='auto',
                                           iterations=400,
                                           passes=20,
                                           eval_every= None)

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [11]:
from pprint import pprint
pprint(lda_model.print_topics())
#To print the % of topics a document is
doc_lda = lda_model[corpus]

[(0,
  '0.074*"case" + 0.073*"death" + 0.073*"new" + 0.031*"report" + '
  '0.031*"number" + 0.021*"infect" + 0.020*"break" + 0.018*"counti" + '
  '0.017*"confirm" + 0.016*"million"'),
 (1,
  '0.090*"test" + 0.038*"patient" + 0.034*"news" + 0.033*"hospit" + '
  '0.031*"posit" + 0.022*"doctor" + 0.017*"nurs" + 0.016*"decreas" + '
  '0.015*"result" + 0.015*"test_posit"'),
 (2,
  '0.031*"live" + 0.028*"today" + 0.027*"updat" + 0.022*"join" + 0.019*"dstv" '
  '+ 0.016*"april" + 0.016*"data" + 0.014*"dr" + 0.014*"watch" + '
  '0.013*"inform"'),
 (3,
  '0.051*"home" + 0.032*"stay" + 0.026*"keep" + 0.023*"safe" + '
  '0.022*"research" + 0.011*"anyon" + 0.011*"stayhom" + 0.010*"stay_home" + '
  '0.010*"hand" + 0.009*"rais"'),
 (4,
  '0.037*"thi" + 0.023*"ha" + 0.019*"peopl" + 0.017*"covid" + 0.016*"u" + '
  '0.013*"get" + 0.012*"wa" + 0.012*"time" + 0.010*"one" + 0.010*"like"'),
 (5,
  '0.042*"dure" + 0.035*"pandem" + 0.027*"help" + 0.024*"amp" + 0.017*"govern" '
  '+ 0.016*"busi" + 0.016*"supp